## (1) Financial Data Science: Realized Volatility

## Step 1

In [2]:
pip install skfolio yahooquery

In [ ]:
import datetime as dt
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
from skfolio.datasets import load_sp500_dataset, load_sp500_implied_vol_dataset
from skfolio.preprocessing import prices_to_returns
import datetime
import math
from yahooquery import Ticker

In [ ]:
#SOME FUNCTIONS FROM THE APPENDIX

def mu_hat_calculator(data):
  N = len(data)
  T = (data.index[-1] - data.index[0]).days / 365.0
  mu_hat = np.log(stock.iloc[-1] / stock.iloc[0]) / T

  return mu_hat


def sigma_hat_calculator(data):
   N = len(data)
   T = (data.index[-1] - data.index[0]).days / 365.0
   mu_hat = mu_hat_calculator(data)
   values = data.values
   indices = (data.index - data.index[0]).days / 365.0

   summation = 0.0
   for i in range(N - 1):
    time_difference = indices[i + 1] - indices[i]
    one_step_return = (values[i + 1] - values[i]) / values[i]
    summation += (one_step_return ** 2) / time_difference

   sigma_squared_hat = summation / (N -1) - ( T / (N -1)) * mu_hat**2

   return sigma_squared_hat

## Step 2

In [ ]:
def download_prices_stooq(ticker: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Download daily OHLCV from Stooq using pandas_datareader.
    For US stocks, Stooq often uses the format ’AAPL.US’.
    """
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    candidates = [
        ticker,
        ticker.upper(),
        f"{ticker.upper()}.US",
        f"{ticker.upper()}.US"
    ]

    last_err = None
    for t in candidates:
        try:
            df = pdr.DataReader(t, "stooq", start, end)
            if df is not None and not df.empty:
                df = df.sort_index()
                df.index = pd.to_datetime(df.index)
                return df
        except Exception as e:
            last_err = e

    raise RuntimeError(
        f"Failed to download data for {ticker} from Stooq. "
        f"Tried: {candidates}. Last error: {last_err}"
    )


# -----------------------------
# Parameters
# -----------------------------
ticker = "AAPL"
start_date = "2010-01-01"
end_date = dt.date.today().strftime("%Y-%m-%d")

# -----------------------------
# Download data (Stooq)
# -----------------------------
data = download_prices_stooq(ticker, start_date, end_date)

# Use Close (Stooq doesn’t always provide Adj Close)
stock = data["Close"].astype(float)
stock


,Close
Date,
2010-01-04,6.41840
2010-01-05,6.42940
2010-01-06,6.32677
2010-01-07,6.31559
2010-01-08,6.35809
...,...
2026-02-06,278.12000
2026-02-09,274.62000
2026-02-10,273.68000


## Step 3

- Select a ticker and a start and end date (ideally a long time window) to plot estimators
for the mean and realized variance using the adjusted closing price.
- Compute the classical historical mean and volatility estimators (as referenced in equa-
tions (8) and (13) in your notes).
- Implement one alternative volatility estimator of your choice (e.g., Parkinson or Garman–
Klass) and compare it with the classical volatility estimator.
- Compute a rolling window estimator using a 30-day window. Plot the results for all
estimators and explain your findings.

In [ ]:
# "Classical estimators as in the notes"
mu_hat = mu_hat_calculator(stock)
sigma_squared_hat = sigma_hat_calculator(stock)
print(mu_hat, sigma_squared_hat)

0.23006506357203205 0.09600192250879819


Select one of the S&P 500 stocks and compare the realized variance estimators
with the time series of implied volatilities for that ticker.

In [ ]:
prices = load_sp500_dataset()
implied_vol = load_sp500_implied_vol_dataset()
X = prices_to_returns(prices)
X = X.loc["2010":]
implied_vol.tail()

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
2022-12-21,0.343749,0.512436,0.314423,0.410686,0.301621,0.337233,0.291449,0.169588,0.276606,0.178861,0.287204,0.220556,0.312139,0.186306,0.258662,0.191366,0.557777,0.244674,0.219240,0.306320
2022-12-22,0.357983,0.541554,0.323598,0.418840,0.317364,0.352863,0.304409,0.182946,0.290541,0.188259,0.289049,0.226861,0.327944,0.194996,0.262856,0.196550,0.575974,0.251015,0.226632,0.319773
2022-12-23,0.354818,0.531073,0.316493,0.400008,0.304423,0.348108,0.295155,0.174784,0.283626,0.182933,0.285319,0.218791,0.323790,0.189222,0.256927,0.193781,0.563745,0.246426,0.219599,0.306954
2022-12-27,0.366786,0.545771,0.318301,0.405272,0.307501,0.355508,0.301645,0.180025,0.286293,0.184747,0.295695,0.225937,0.334950,0.190844,0.265863,0.197682,0.570953,0.254200,0.225493,0.311402
2022-12-28,0.388422,0.549840,0.322165,0.429541,0.315993,0.364447,0.308722,0.179856,0.295815,0.192887,0.295466,0.228423,0.340077,0.196283,0.267425,0.208188,0.581149,0.253623,0.236076,0.323964


## Step 4

## (a)

In [ ]:
def fetch_spx_close(start_date, end_date):
    """
    Fetch an SPX-like daily close series.
    Primary: FRED SP500
    Fallback: Stooq ˆSPX
    Returns a DataFrame with a ’Close’ column and DatetimeIndex.
    """
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    # Try FRED first
    try:
        df = pdr.DataReader("SP500", "fred", start, end)
        df = df.rename(columns={"SP500": "Close"}).dropna()
        df.index = pd.to_datetime(df.index)
        return df
    except Exception:
        pass

    # Fallback: Stooq
    df = pdr.DataReader("ˆSPX", "stooq", start, end)
    df = df.sort_index()
    df.index = pd.to_datetime(df.index)
    return df[["Close"]].dropna()

def fetch_vix_close(start_date, end_date):
    """
    Fetch daily VIX close series.
    Primary: FRED VIXCLS Fallback: Stooq VI.F (VIX)
    Returns a DataFrame with a ’Close’ column and DatetimeIndex.
    """
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    # Try FRED first
    try:
        df = pdr.DataReader("VIXCLS", "fred", start, end)
        df = df.rename(columns={"VIXCLS": "Close"}).dropna()
        df.index = pd.to_datetime(df.index)
        return df
    except Exception:
        pass

    # Fallback: Stooq (try common symbols)
    for sym in ["VI.F", "vi.f", "ˆVIX", "ˆvix"]:
        try:
            df = pdr.DataReader(sym, "stooq", start, end)
            df = df.sort_index()
            df.index = pd.to_datetime(df.index)
            if "Close" in df.columns and not df.empty:
                return df[["Close"]].dropna()
        except Exception:
            continue

    raise ValueError("No VIX data found from FRED or Stooq.")


spx_symbol = "ˆSPX"
today = pd.to_datetime("2025-03-05")  # Keep this fixed in your implementation
end_date = today
start_date = end_date - datetime.timedelta(days=365)

spx_data = fetch_spx_close(start_date, end_date)
if spx_data.empty:
    raise ValueError("No SPX data found from FRED/Stooq for this window.")

lastBusDay = spx_data.index[-1]
S0 = float(spx_data["Close"].iloc[-1])  # spot/closing price

# Fetch VIX in a short window after lastBusDay (first available close)
vix_data = fetch_vix_close(lastBusDay, lastBusDay + datetime.timedelta(days=30))
if vix_data.empty:
    raise ValueError("No VIX data found from FRED/Stooq for this window.")

vix_market = float(vix_data["Close"].iloc[0])

# Fixed inputs (keep these fixed in your implementation)
T = 30 / 365.0
r = 0.02
F0 = S0 * math.exp(r * T)  # forward approximation

print("Last Bus Day:", lastBusDay)
print("S0:", S0)
print("VIX market close:", vix_market)
print("F0:", F0)


Last Bus Day: 2025-03-05 00:00:00
S0: 5842.63
VIX market close: 21.93
F0: 5852.242221579255


## (b)

In [ ]:
#Some issues with Ticker (as we are already using that in some earlier part of the assignment)
#We directly download the files from Canvas instead.

calls = pd.read_csv("Call_option_data_2025-04-03_final.csv")
puts = calls = pd.read_csv("Put_option_data_2025-04-03_final.csv")

option_chain = {
    "calls": calls,
    "puts": puts

}

##Step 5
Compute the estimated VIX using the estimator V IXt (referenced as equation (18) in your
notes) and compare it with the CBOE-quoted VIX.

In [ ]:
#TO DO: ADD CODE

## Step 6
Plot the historical estimated realized variances from Step 3 alongside the VIX time series.
Perform statistical analyses (such as correlation or cointegration tests) to assess the relation-
ship between the time series.

In [ ]:
#TO DO: ADD CODE

## Step 7
Run regression analyses between SPX returns and the VIX index, as well as between SPX
returns and the historical realized variance estimator. Discuss your observations.
Hint: The variations of the stock index are typically negatively correlated with variations of
the VIX index, whereas the correlation with historical volatility variations may differ.

In [ ]:
#TO DO: ADD CODE

## (2) Hedging: Volatility Mismatch [8p]

## Step 1

Matching Volatility: Conduct an experiment where the volatility in the stock price process
matches the volatility used in the delta computation (i.e., both set to 20%). Vary the frequency
of hedge adjustments (from daily to weekly) and explain the results.

In [ ]:
#TO DO: ADD CODE


## Step 2

Mismatched Volatility: Perform numerical experiments where the volatility in the stock
price process does not match the volatility used in the delta valuation. Run computational
experiments for various levels of volatility and discuss the outcomes.

In [ ]:
#TO DO: ADD CODE

## Step 3: See Assignment

In [ ]:
#TO DO: ADD CODE

## (3) Hedging with Real SPX Option Data

## Step 1

In [ ]:
## Data Loading
data_SPX = pd.read_csv("option20230201_20230228.csv") # Load SPX option quote data
data_SPX = data_SPX[["date","exdate","symbol","strike_price","best_bid","best_offer","impl_volatility","delta","cp_flag"]] # Discard irrelevant features
print("Raw data observation count: ", data_SPX.shape[0]) # Reporting..


## Data Preprocessing
calls = data_SPX[data_SPX["cp_flag"] == "C"].copy() # Keep calls only
print("Observation count after filtering to calls: ", calls.shape[0]) # Reporting..

calls["date"] = pd.to_datetime(calls["date"]) # Convert entry type for "date features" to datetime64[ns] for consistency
calls["exdate"] = pd.to_datetime(calls["exdate"])
calls["strike"] = calls["strike_price"] / 1000 # Rescale strikes to match index-point units, i.e. division by 1000
calls.drop(columns="strike_price", inplace=True)
calls["midquote"] = (calls["best_bid"] + calls["best_offer"]) / 2 # Construct midquote
# print("\nMidquote (V_t) missing rate: ", calls["midquote"].isna().mean())
print("\nMidquote (V_t) missing / incorrect data rate: ",
  (
      calls["best_bid"].isna() | # Missing entries
      calls["best_offer"].isna() |
      (calls["best_bid"] < 0) | # Best bid may be 0 (no buyers), but should never be negative
      (calls["best_offer"] < 0) | # Best offer should always be positive, as the value of a European call option is always at least 0, and giving away such an option for free is not sensible
      (calls["best_bid"] >= calls["best_offer"]) # This should result in a market order, clearing the associated bid/offer
  ).mean())
print("Implied volatility (sigma_mkt) missing / incorrect data rate: ", (calls["impl_volatility"].isna() | (calls["impl_volatility"] <= 0)).mean())
print("Delta missing data rate: ", calls["delta"].isna().mean())

calls = calls.dropna(subset=["impl_volatility"]) # Remove observations with missing (NaN) or invalid (<=0) implied volatility
calls = calls[calls["impl_volatility"] > 0]
print("\nFinal observation count after preprocessing: ", calls.shape[0]) # Reporting..
print("\nUnique trading dates (after preprocessing): ", calls["date"].nunique())
print("Unique expiries (after preprocessing): ", calls["exdate"].nunique())

print("\nFinal columns: ", calls.columns)


## Further Data Augmenting..
# spx_close = fetch_spx_close("2023-02-01", "2023-03-01")

Raw data observation count:  175301
Observation count after filtering to calls:  175301

Midquote (V_t) missing / incorrect data rate:  0.0
Implied volatility (sigma_mkt) missing / incorrect data rate:  0.12306261801130627
Delta missing data rate:  0.12306261801130627

Final observation count after preprocessing:  153728

Unique trading dates (after preprocessing):  19
Unique expiries (after preprocessing):  67

Final columns:  Index(['date', 'exdate', 'symbol', 'best_bid', 'best_offer', 'impl_volatility',
       'delta', 'cp_flag', 'strike', 'midquote'],
      dtype='object')


In [ ]:
# data_SPX = pd.read_csv("option20230201_20230228.csv")

# # SPX Close does not seem to be present --> This makes sense because data_SPX contains
# # only the historical SPX call option quotes, not the historical SPX price data itself
# # ==> Have to source this manually..
# # print(data_SPX.columns)
# data_SPX = data_SPX[["date","exdate","symbol","strike_price","best_bid","best_offer","impl_volatility","delta","cp_flag", "last_date"]]
# # print(data_SPX)

# calls = data_SPX[data_SPX["cp_flag"] == "C"].copy()
# calls["date"] = pd.to_datetime(calls["date"])
# calls["exdate"] = pd.to_datetime(calls["exdate"])
# calls["last_date"] = pd.to_datetime(calls["last_date"])
# calls["strike_price"] = calls["strike_price"] / 1000

# calls["mid_quote"] = (calls["best_bid"] + calls["best_offer"]) / 2

# calls = calls.dropna(subset=["impl_volatility"])
# calls = calls[calls["impl_volatility"] > 0]

# # updated_data = fetch_spx_close("2023-02-27", "2023-03-01")
# # print(updated_data.shape)

# print("Original number of rows: ", data_SPX.shape[0], "Final number of rows: ", calls.shape[0])
# print("Unique trading dates and expiries respectively: ", calls["date"].nunique(), calls["exdate"].nunique())
# # print(calls)

# print("Missing rate V_t: ", calls["mid_quote"].isna().mean())
# print("Missing rate sigma_mkt: ", calls["impl_volatility"].isna().mean())
# print("Missing rate delta: ", calls["delta"].isna().mean())

# print("Final columns: ", calls.columns)

# ######## NOTES//<

# print(data_SPX.columns)

# optionid = pd.read_csv("option20230201_20230228.csv")["optionid"]
# print(len(optionid))
# print(len(optionid.unique()))

# print('-'*25)
# # NOTE: We assume that the asset price process evolves exclusively during trading days (252) total,
# # so that the remaining 365-252=113 calender days essentially vanish. These are not
# # taken into account in estimates and other calculations.

# spx_close_data = fetch_spx_close("2023-02-01", "2023-03-01")
# # print(spx_close_data.shape)
# # print(spx_close_data)
# # We get a total of 20 entries, totalling 20 "trading days"!


# # type(spx_close_data.index) # Unique date entries in spx_close_data
# # list(data_SPX['date'].unique()) # Unique date entries in data_SPX
# #
# for date in list(data_SPX['date'].unique()):
#   if date not in spx_close_data.index:
#     print(date)
# # It would appear all dates in data_SPX are represented in the fetched spx_close_data,
# # Hence we can merge data_SPX and spx_close_data into one dataframe object.

# data_SPX["date"] = pd.to_datetime(data_SPX["date"]) # First change entry type to pd.datetime, to ensure type consistency for the merge
# augmented_data_SPX = pd.merge(data_SPX, spx_close_data, how='inner', left_on='date', right_index=True)
# print(spx_close_data)
# augmented_data_SPX

# # And to round off, we also sort the entries based on chronological order???
# # augmented_data_SPX = augmented_data_SPX.sort_values(by='date')
# # augmented_data_SPX

# ######## >//END

Original number of rows:  175301 Final number of rows:  153728
Unique trading dates and expiries respectively:  19 67
Missing rate V_t:  0.0
Missing rate sigma_mkt:  0.0
Missing rate delta:  0.0
Final columns:  Index(['date', 'exdate', 'symbol', 'strike_price', 'best_bid', 'best_offer',
       'impl_volatility', 'delta', 'cp_flag', 'last_date', 'mid_quote'],
      dtype='object')
Index(['date', 'exdate', 'symbol', 'strike_price', 'best_bid', 'best_offer',
       'impl_volatility', 'delta', 'cp_flag', 'last_date'],
      dtype='object')
175301
12735
-------------------------
              Close
DATE               
2023-02-01  4119.21
2023-02-02  4179.76
2023-02-03  4136.48
2023-02-06  4111.08
2023-02-07  4164.00
2023-02-08  4117.86
2023-02-09  4081.50
2023-02-10  4090.46
2023-02-13  4137.29
2023-02-14  4136.13
2023-02-15  4147.60
2023-02-16  4090.41
2023-02-17  4079.09
2023-02-21  3997.34
2023-02-22  3991.05
2023-02-23  4012.32
2023-02-24  3970.04
2023-02-27  3982.24
2023-02-28  3970.15

,date,exdate,symbol,strike_price,best_bid,best_offer,impl_volatility,delta,cp_flag,last_date,Close
0,2023-02-01,2023-02-17,SPX 230217C1000000,1000000,3111.0,3118.5,NaN,NaN,C,2023-01-26,4119.21
1,2023-02-01,2023-02-17,SPX 230217C1200000,1200000,2911.5,2919.1,NaN,NaN,C,2022-07-28,4119.21
2,2023-02-01,2023-02-17,SPX 230217C1300000,1300000,2811.6,2819.7,NaN,NaN,C,NaN,4119.21
3,2023-02-01,2023-02-17,SPX 230217C1400000,1400000,2711.9,2719.4,NaN,NaN,C,NaN,4119.21
4,2023-02-01,2023-02-17,SPX 230217C1500000,1500000,2612.2,2619.8,NaN,NaN,C,2023-01-31,4119.21
...,...,...,...,...,...,...,...,...,...,...,...
175296,2023-02-28,2023-12-29,SPXW 231229C5100000,5100000,8.7,9.0,0.137016,0.045269,C,2023-02-28,3970.15
175297,2023-02-28,2023-12-29,SPXW 231229C5200000,5200000,6.1,6.5,0.138055,0.033419,C,2023-02-24,3970.15
175298,2023-02-28,2023-12-29,SPXW 231229C5300000,5300000,4.5,4.9,0.140171,0.025474,C,2023-02-16,3970.15
175299,2023-02-28,2023-12-29,SPXW 231229C5400000,5400000,3.4,3.8,0.142722,0.019789,C,2023-02-28,3970.15


In [ ]:
#Some notation
calls["days_to_expiration"] = (calls["exdate"] - calls["date"]).dt.days
calls["year_fraction"] = calls["days_to_expiration"] / 365.0


def epsilon_calculator(mid_quote, delta, S_t):
  #delta_S_t = S_t.diff
  delta_V_t = mid_quote.diff()
  #epsilon = delta_V_t - delta * delta_S_t

  #return epsilon

def SSE_calculator(epsilon):
  SSE_delta = 0.0
  for t in epsilon.index:
    SSE_delta += epsilon.loc[t] ** 2

  return SSE_delta

  def gain_calculator(epsilon_A, epsilon_B):
    SSE_delta_A, SSE_delta_B = SSE_calculator(epsilon_A), SSE_calculator(epsilon_B)
    return 1 - SSE_A / SSE_B

## Step 2

In [ ]:
#TO DO: ADD CODE

##Step 3

In [ ]:
start = "2023-02-01"
end = "2023-03-02"

d1 = pd.Timestamp(start).strftime("%Y%m%d")
d2 = pd.Timestamp(end).strftime("%Y%m%d")

url = f"https://stooq.com/q/d/l/?s=^SPX&d1={d1}&d2={d2}&i=d"
df = pd.read_csv(url, parse_dates=["Date"]).sort_values("Date")

SPX_price_series = df["Close"]


## Step 4

In [ ]:
url = (
"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/"
"daily-treasury-rate-archives/par-yield-curve-rates-2020-2023.csv"
)

treasury = pd.read_csv(url)
treasury["date"] = pd.to_datetime(treasury["date"]).dt.normalize()


## Step 5

In [ ]:
#TO DO: ADD CODE

##Step 6

In [ ]:
#TO DO: ADD CODE

## Step 7

In [ ]:
#TO DO: ADD CODE